<a href="https://colab.research.google.com/github/karoldem/comicfinder/blob/master/comicfinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dear reader,

what you see is some unordered fragments of code to be stolen from.

It is for webscraping comics pages, performing OCR on images and saving results to a file.

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
from requests import get
from bs4 import BeautifulSoup
from pytesseract import image_to_string
from json import load, dump
from io import BytesIO
import re

try:
    from PIL import Image
except ImportError:
    import Image

savefile = True
if savefile:
    from google.colab import drive
    drive.mount('/content/gdrive')

class comicFinder:

    databaseSource = '/content/gdrive/My Drive/comictranscriotion.txt'
    reversedDatabaseSource = '/content/gdrive/My Drive/comictranscriotionreversed.txt'
    blablahfile = '/content/gdrive/My Drive/blahblah.txt'

    def appendToJSONFile(self,newKey, newValue):

        with open(self.databaseSource, "r") as jsonFile:
            data = load(jsonFile)

        data[ newKey ] = newValue

        with open(self.databaseSource, "w") as jsonFile:
            dump(data, jsonFile)

    def blahblahblah(self):
        with open(self.databaseSource, "r") as jsonFile:
            data = load(jsonFile)

        toreturn = {}

        flag = True

        for i in data:
            match = re.compile(r'[A-Za-z]+')
            match = match.findall(data[i])
            for j in match:
                if j in toreturn: print('it is '+ i + ' ' + j + ' ' + str(match))
                else: print("no, it's not " + i + ' ' + j + ' ' + str(match))


        with open(self.blablahfile, "w") as jsonFile:
            dump(toreturn, jsonFile)



    def extractText(self):

      lastComicURL = self.lastComicChecked()
      url = self.lastComicPosted()

      while not url == lastComicURL:

          comicElem = []

          for i in range(2):
            res = get(url)
            res.raise_for_status()
            soup = BeautifulSoup(res.text)
            comicElem = self.getComicElem(soup)
            if comicElem != []: break

          if not comicElem == []:
              try:
                    comicUrl = self.getComicUrl(comicElem)
                    res = get(comicUrl)
                    res.raise_for_status()
                    img = Image.open(BytesIO(res.content))

                    comicText = image_to_string(img)
                    self.appendToJSONFile(url,comicText.capitalize())

              except Exception as e:
                    print (e)

          try:
            url = self.selectNextComic(soup)
          except:
              break


Mounted at /content/gdrive


In [ ]:
blah = comicFinder()
print("")
print(blah.blahblahblah())
print("")

{2: 3, 4: 5}

In [ ]:
import re

databaseSource = '/content/gdrive/My Drive/comictranscriotion.txt'

with open(databaseSource, "r") as jsonFile:
    data = load(jsonFile)

wordDict = {}

for link in data:

    for word in re.findall(r"[^\W\d_]+", data[link]):
        if word in wordDict:
            wordDict[word].add(link)
        else:
            wordDict[word] = {link}


In [ ]:
class xkcd (comicFinder):

    lastComicCheckedFakeFile = 'http://xkcd.com/2596/'

    lastComicChecked = lambda self: self.lastComicCheckedFakeFile
    def updateLastComicChecked (self, url): self.lastComicCheckedFakeFile = url

    selectNextComic = lambda self, x :'http://xkcd.com' +  x.select('a[rel="prev"]')[0].get('href')
    selectPreviousComic = lambda self, x :'http://xkcd.com' +  x.select('a[rel="next"]')[0].get('href')
    getComicElem = lambda self, x : x.select('#comic img')
    getComicUrl = lambda self, x: 'http:' + x[0].get('src')

    def lastComicPosted(self):
        res = get('http://xkcd.com/')
        soup = BeautifulSoup(res.text)
        url = self.selectNextComic(soup)
        res = get(url)
        soup = BeautifulSoup(res.text)
        url = self.selectPreviousComic(soup)
        self.updateLastComicChecked(url)
        return url

In [ ]:
x = xkcd()
x.extractText()
x.lastComicCheckedFakeFile

'http://xkcd.com/2899/'

# Archive

In [ ]:
import requests, os, bs4
from io import BytesIO
import pytesseract
import shutil
import os
import random
import json
import operator

try:
    from PIL import Image
except ImportError:
    import Image

savefile = False
if savefile:
    from google.colab import drive
    drive.mount('/content/gdrive')

def appendToJSONFile(newKey, newValue):

    with open('/content/gdrive/My Drive/comictranscriotion.txt', "r") as jsonFile:
        data = json.load(jsonFile)

    data[ newKey ] = newValue

    with open('/content/gdrive/My Drive/comictranscriotion.txt', "w") as jsonFile:
        json.dump(data, jsonFile)


class comicOCRer:


    def __init__(self, pageUrl, firstComicURL, lastComicURL, findThoseWordsInText):

        self.pageUrl = pageUrl
        self.firstComicURL = firstComicURL
        self.lastComicURL =  lastComicURL
        self.findThoseWordsInText = findThoseWordsInText

        if pageUrl == 'http://xkcd.com':
            self.selectNextComic = lambda x : 'http://xkcd.com' + x.select('a[rel="prev"]')[0].get('href')
            self.selectPreviousComic = lambda x : 'http://xkcd.com' + x.select('a[rel="next"]')[0].get('href')
            self.getComicElem = lambda x : x.select('#comic img')
            self.getComicUrl = lambda x: 'http:' + x[0].get('src')
            self.checkLastComic = self.checkLastComicXkcd

        if pageUrl == 'http://smbc-comics.com':
            self.selectNextComic = lambda x : x.select('a[rel="prev"]')[0].get('href')
            self.selectPreviousComic = lambda x : x.select('a[rel="next"]')[0].get('href')
            self.getComicElem = lambda x : x.select('#cc-comicbody img')
            self.getComicUrl = lambda x: x[0].get('src')
            self.checkLastComic = self.checkLastComicXkcd


    def run(self):
        return self.extractText(
                self.pageUrl,
                self.firstComicURL,
                self.lastComicURL,
                self.findThoseWordsInText,
                self.selectNextComic,
                self.getComicElem)

    turnOffOCR = False

    def checkLastComicXkcd(self):
        res = requests.get(self.pageUrl)
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text)
        url = self.selectNextComic(soup)
        res = requests.get(url)
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text)
        return self.selectPreviousComic(soup)

    def extractText(self, pageUrl, firstComicURL, lastComicURL, findThoseWordsInText, selectNextComic, getComicElem):

      wordCounter = {}
      transcription = {}
      url = firstComicURL

      while not url == lastComicURL:
          comicElem = []
          for i in range(2):
            res = requests.get(url)
            res.raise_for_status()
            soup = bs4.BeautifulSoup(res.text)
            comicElem = self.getComicElem(soup)
            if comicElem != []: break

          if comicElem == []:
              wordCounter[url] = 0
          else:
              try:
                    comicUrl = self.getComicUrl(comicElem)
                    res = requests.get(comicUrl)
                    res.raise_for_status()
                    img = Image.open(BytesIO(res.content))

                    if not self.turnOffOCR:
                      comicText = pytesseract.image_to_string(img)
                      if savefile:
                        appendToJSONFile(url,comicText.capitalize())
                      transcription[url] = comicText.capitalize()

                      for i in findThoseWordsInText:
                        if i in comicText:

                          try:
                            wordCounter[url] +=1
                          except:
                            wordCounter[url] = 1
                    else:
                      print (url)

              except Exception as e:
                    wordCounter[url] = 0
                    print(e)


          try:
            url = selectNextComic(soup)
          except:
              break


      wordCounter = sorted(wordCounter.items(), key=operator.itemgetter(1), reverse=True)
      return wordCounter

In [ ]:
comicOCRer(
    pageUrl = 'http://smbc-comics.com',
    firstComicURL = 'http://smbc-comics.com',
    lastComicURL = 'https://www.smbc-comics.com/comic/i',
    findThoseWordsInText = ['direct', 'cultural', 'some', 'countries']
    ).run()

[('https://www.smbc-comics.com/comic/cast', 1),
 ('https://www.smbc-comics.com/comic/a-city-on-mars-2', 1),
 ('https://www.smbc-comics.com/comic/rite', 1),
 ('https://www.smbc-comics.com/comic/dev', 1),
 ('https://www.smbc-comics.com/comic/the-painting', 1),
 ('https://www.smbc-comics.com/comic/bea-wolf', 1),
 ('https://www.smbc-comics.com/comic/citation', 1),
 ('https://www.smbc-comics.com/comic/circling', 1),
 ('https://www.smbc-comics.com/comic/priority', 1),
 ('https://www.smbc-comics.com/comic/freaks', 1)]

In [ ]:
comicOCRer(
    pageUrl = 'http://xkcd.com',
    firstComicURL = 'https://xkcd.com',
    lastComicURL = 'hsdfghjkiuytgf',
    findThoseWordsInText = []
    ).run()